In [4]:
import os
if not os.path.exists('/content/AI574'):
    !git clone https://github.com/roshcs/AI574.git
%cd /content/AI574
!git pull

/content/AI574
Already up to date.


In [5]:
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

from google.colab import drive
drive.mount('/content/drive')


# Create a persistent project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/AI574_Multi_Domain_Agent'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/data', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/models', exist_ok=True)
print(f'Project directory: {PROJECT_DIR}')


Mounted at /content/drive
Project directory: /content/drive/MyDrive/AI574_Multi_Domain_Agent


In [6]:
# Action: Run this cell to install all packages:

# %%capture
!pip install -q \
  "langchain>=0.3.0" "langchain-core>=0.3.0" "langgraph>=0.2.0" "langchain-text-splitters>=0.2.0" \
  "keras>=3.0" "keras-hub>=0.17.0" \
  "sentence-transformers>=3.0.0" "transformers>=4.45,<5.0" "chromadb>=0.5.0" \
  "PyPDF2>=3.0.0" "arxiv>=2.1.0" "pydantic>=2.0"

!pip install -q "jax[cuda12]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 121.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 46.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 124.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6

In [7]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

# Verify
import keras
print(f'Keras backend: {keras.backend.backend()}')
print(f'Keras version: {keras.__version__}')

Keras backend: jax
Keras version: 3.10.0


In [8]:
import langchain
import langgraph
import keras_hub
import chromadb
from sentence_transformers import SentenceTransformer
import jax
print('All imports successful!')
print(f'JAX devices: {jax.devices()}')

All imports successful!
JAX devices: [CudaDevice(id=0)]


In [9]:
import os
import sys

# PROJECT_DIR is already defined
PROJECT_ROOT = os.path.join(PROJECT_DIR, 'project')

dirs = [
    'config',
    'foundation',
    'ingestion',
    'rag_core',
    'agents',
    'orchestration',
    'evaluation',
    'data/industrial',
    'data/recipes',
]

# Create directories
for d in dirs:
    os.makedirs(os.path.join(PROJECT_ROOT, d), exist_ok=True)

# Create __init__.py in every package directory
packages = [
    'config',
    'foundation',
    'ingestion',
    'rag_core',
    'agents',
    'orchestration',
    'evaluation'
]

for pkg in packages:
    init_file = os.path.join(PROJECT_ROOT, pkg, '__init__.py')
    open(init_file, 'a').close()

# Add project root to Python path
sys.path.insert(0, PROJECT_ROOT)

print('Project structure created at:', PROJECT_ROOT)

Project structure created at: /content/drive/MyDrive/AI574_Multi_Domain_Agent/project


In [10]:
import os, sys

# PROJECT_DIR already defined elsewhere
PROJECT_ROOT = os.path.join(PROJECT_DIR, "project")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT =", PROJECT_ROOT)
print("foundation exists?", os.path.isdir(os.path.join(PROJECT_ROOT, "foundation")))
print("init exists?", os.path.isfile(os.path.join(PROJECT_ROOT, "foundation", "__init__.py")))
print("files:", os.listdir(os.path.join(PROJECT_ROOT, "foundation")))

PROJECT_ROOT = /content/drive/MyDrive/AI574_Multi_Domain_Agent/project
foundation exists? True
init exists? True
files: ['__pycache__', '__init__.py', 'embedding_service.py', 'llm_wrapper.py', 'vector_store.py']


In [11]:
import os, sys
from config.settings import CONFIG, LLMConfig
from foundation.llm_wrapper import KerasHubChatModel
from foundation.embedding_service import EmbeddingService
from foundation.vector_store import VectorStoreService

# Print configuration
print(f"LLM: {CONFIG.llm.preset}")
print(f"Embeddings: {CONFIG.embedding.model_name}")
print(f"Domains: {CONFIG.supervisor.domains}")

from google.colab import userdata
import os

os.environ["KAGGLE_USERNAME"] = "roshcs"
os.environ["KAGGLE_KEY"] = "781502c08d14d778d3d23d4e58ca0751"

# os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
# os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

LLM: gemma3_instruct_12b
Embeddings: thenlper/gte-large
Domains: ['industrial', 'recipe', 'scientific']


In [12]:
# Now try loading
from foundation.llm_wrapper import KerasHubChatModel
from config.settings import CONFIG

llm = KerasHubChatModel(config=CONFIG.llm)
print("✅ LLM loaded")


✅ LLM loaded


In [13]:
from foundation.embedding_service import EmbeddingService

embedder = EmbeddingService()

test_vec = embedder.embed_query("PLC fault code")
print(f"✅ Embedding dimension: {len(test_vec)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

✅ Embedding dimension: 1024


In [14]:
from foundation.vector_store import VectorStoreService

vs = VectorStoreService(embedding_service=embedder)
print("✅ Vector store initialized")
print(vs.get_all_stats())

✅ Vector store initialized
[{'domain': 'industrial', 'collection_name': 'industrial_knowledge', 'document_count': 0}, {'domain': 'recipe', 'collection_name': 'recipe_knowledge', 'document_count': 0}, {'domain': 'scientific', 'collection_name': 'scientific_knowledge', 'document_count': 0}]


In [15]:
from ingestion.index_builder import IndexBuilder
from ingestion.document_loader import DocumentLoader
from ingestion.chunking_pipeline import ChunkingPipeline

builder = IndexBuilder(vector_store=vs)

# ── Industrial: Rockwell Automation / Allen-Bradley ──
sample_industrial = [
    """Fault Code F002 - Allen-Bradley PowerFlex 525 VFD: Auxiliary Input.
    Description: An external fault has been detected via the auxiliary input terminal.
    Possible Causes:
    1. External device connected to auxiliary input is reporting a fault condition
    2. Wiring issue on the auxiliary digital input terminal
    3. Incorrect parameter configuration for the auxiliary input function
    Troubleshooting Steps:
    1. Check the external device connected to the auxiliary input for faults
    2. Verify wiring connections on terminal block TB2
    3. Check parameter A070 [Fault Config 2] for correct auxiliary input configuration
    4. Inspect for damaged or loose wiring on the digital input terminals
    Resolution: Clear the external fault condition and reset the drive by cycling the Enable input or pressing the Stop/Reset key on the HIM.""",

    """Fault Code F004 - Allen-Bradley PowerFlex 525 VFD: Undervoltage.
    Description: The DC bus voltage has dropped below the undervoltage trip level.
    Possible Causes:
    1. Input power supply voltage too low or momentary power dip
    2. Incoming power supply fuses blown or circuit breaker tripped
    3. Loose connections on input power terminals R/L1, S/L2, T/L3
    4. DC bus capacitors degraded (check capacitor health indicator LED)
    Troubleshooting Steps:
    1. Measure incoming line voltage at drive terminals - must be within nameplate rating +/-10%
    2. Check all three phases for voltage balance (max 3% imbalance)
    3. Inspect and tighten input power terminal connections (torque to 1.4 Nm)
    4. Check parameter A531 [DC Bus Voltage] for current reading
    5. Review fault log in parameter A700-A706 for fault history and timestamps
    Resolution: Restore proper input voltage. If capacitors are degraded, replace the drive.
    SAFETY: Disconnect and lockout/tagout all power sources before inspecting terminals.
    Wait 5 minutes for DC bus capacitors to discharge before servicing.""",

    """Fault Code F005 - Allen-Bradley PowerFlex 525 VFD: Overvoltage.
    Description: The DC bus voltage has exceeded the overvoltage trip level.
    Possible Causes:
    1. Input line voltage exceeds drive nameplate rating
    2. Excessive regenerative energy from motor deceleration (overhauling load)
    3. Deceleration time too short for the load inertia
    4. Dynamic brake resistor circuit failure or incorrect sizing
    Troubleshooting Steps:
    1. Measure input voltage - must not exceed 528 VAC for 480V class drives
    2. Increase deceleration time in parameter A092 [Decel Time 1]
    3. Enable bus regulator: set parameter A540 [Bus Reg Mode] to option 1 (Enabled)
    4. If using dynamic braking, check DB resistor connections and resistance value
    5. Check parameter A531 [DC Bus Voltage] - nominal is ~650V for 480V input
    Resolution: Reduce input voltage or extend deceleration time. Add DB resistor for overhauling loads.""",

    """Allen-Bradley CompactLogix 5380 Controller - EtherNet/IP Configuration Guide.
    Model: 1769-L33ER CompactLogix 5380
    Prerequisites: RSLogix 5000 v32+ or Studio 5000 Logix Designer v32+
    Step 1: Create new project - select 1769-L33ER controller, revision 32+
    Step 2: Configure controller IP address via USB connection using BOOTP/DHCP tool
    Step 3: Add EtherNet/IP module in I/O Configuration tree
    Step 4: Right-click controller > Properties > set IP address (e.g., 192.168.1.10)
    Step 5: Add remote I/O devices - right-click EtherNet/IP > New Module
    Step 6: Configure Produced/Consumed tags for controller-to-controller communication
    Step 7: Download program and verify connection indicators:
      - OK LED: Solid green = running, Flashing green = program mode
      - ENET LED: Solid green = has connections, Flashing green = no connections
    Common Issue: If ENET LED is off, check IP config and network switch connection.
    Use RSLinx Classic > RSWho to verify controller appears on EtherNet/IP network.""",

    """Allen-Bradley PanelView Plus 7 HMI - Troubleshooting Communication Failures.
    Symptom: PanelView displays "Controller not found" or shows stale data.
    Possible Causes:
    1. Incorrect IP address or subnet mask configuration
    2. EtherNet/IP cable disconnected or faulty
    3. RSLinx Enterprise communication path misconfigured
    4. Controller is in Program mode instead of Run mode
    Troubleshooting Steps:
    1. On PanelView: press Terminal Settings > Networks > verify IP address
    2. Ping the controller IP from PanelView diagnostics screen
    3. In FactoryTalk View Studio: check Communication Setup > verify shortcut path
    4. Ensure controller and HMI are on same subnet (e.g., both 192.168.1.x/24)
    5. Check Ethernet cable and switch port LEDs for link activity
    6. Review controller mode - must be in Run or Remote Run for live data
    Resolution: Correct IP addressing and verify physical network connectivity.
    If using managed switch, verify VLANs are not isolating devices.""",

    """Preventive Maintenance Schedule - Allen-Bradley PowerFlex 755 Drive System.
    Weekly: Check drive status LEDs and HIM display for active alarms
    Monthly: Inspect cooling fans for proper operation, clean air filters
    Quarterly: Verify DC bus voltage (parameter 15 - Bus Voltage), check capacitor
    formation indicator, tighten all power and control terminal connections
    Semi-Annually: Thermal scan of power connections, verify ground fault monitoring,
    backup drive parameters using Connected Components Workbench (CCW)
    Annually: Full parameter backup, firmware version audit against Rockwell
    compatibility matrix, capacitor health test, clean power module heat sinks
    Every 5 Years: Replace cooling fans (recommended life), capacitor reformation
    MTBF: PowerFlex 755 rated at approximately 200,000 hours (28+ years) at 40C ambient.
    Critical Spares to Stock: Cooling fan assembly (KIT-F755FAN), control board fuse,
    fiber optic cables for multi-axis configurations.""",
]

count = builder.index_industrial_texts(sample_industrial, source="rockwell_automation")
print(f"✅ Indexed {count} industrial chunks")

# ── Recipe sample data ──
sample_recipes = [
    """Recipe: Classic Chocolate Chip Cookies
    Prep Time: 45 minutes. Ingredients: 2 1/4 cups flour, 1 tsp baking soda,
    1 tsp salt, 1 cup butter softened, 3/4 cup sugar, 3/4 cup brown sugar,
    2 large eggs, 2 tsp vanilla, 2 cups chocolate chips.
    Egg Substitutes: Use 1/4 cup unsweetened applesauce per egg,
    or 1 mashed banana per egg, or 3 tbsp aquafaba per egg.""",

    """Recipe: Quick Chicken Stir-Fry
    Prep Time: 20 minutes. Ingredients: 1 lb chicken breast, 2 bell peppers,
    1 cup rice, 3 tbsp soy sauce, 1 tbsp sesame oil, 2 cloves garlic, ginger.
    Steps: Cook rice. Slice chicken. Heat oil in wok over high heat.
    Stir-fry chicken 5-6 min. Add vegetables and garlic. Add soy sauce.
    Nutrition: ~450 calories per serving, 35g protein.""",

    """Recipe: Homemade Margherita Pizza
    Prep Time: 90 minutes (including dough rise). Ingredients: 3 cups flour,
    1 packet yeast, 1 cup warm water, 2 tbsp olive oil, 1 tsp salt, 1 tsp sugar,
    1 cup crushed San Marzano tomatoes, 8 oz fresh mozzarella, fresh basil.
    Steps: Mix dough, let rise 1 hour. Preheat oven to 475F. Stretch dough,
    add sauce and cheese. Bake 12-15 min until crust is golden.
    Gluten-Free Alternative: Substitute 3 cups gluten-free flour blend plus
    1 tsp xanthan gum for regular flour.""",
]

recipe_docs = DocumentLoader.from_texts(sample_recipes, "recipe", "demo_recipes")
chunker = ChunkingPipeline()
recipe_chunks = chunker.chunk_documents(recipe_docs, domain="recipe")
count = vs.add_documents("recipe", recipe_chunks)
print(f"✅ Indexed {count} recipe chunks")

# ── Scientific sample data ──
sample_scientific = [
    """Title: Attention Is All You Need
    Authors: Vaswani, Shazeer, Parmar, Uszkoreit, Jones, Gomez, Kaiser, Polosukhin
    Abstract: We propose a new network architecture, the Transformer, based solely
    on attention mechanisms, dispensing with recurrence and convolutions entirely.
    The Transformer allows for significantly more parallelization and achieves
    new state of the art in translation quality. ArXiv ID: 1706.03762""",

    """Title: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
    Authors: Lewis, Perez, Piktus, Petroni, Karpukhin, Goyal, Kuttler, Lewis, Yih, Rocktaschel, Riedel, Kiela
    Abstract: We explore a general-purpose fine-tuning recipe for retrieval-augmented
    generation (RAG) which combines pre-trained parametric and non-parametric memory
    for language generation. ArXiv ID: 2005.11401""",

    """Title: Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection
    Authors: Asai, Wu, Wang, Sil, Hajishirzi
    Abstract: We introduce Self-RAG, a framework that trains a single LM to adaptively
    retrieve passages on demand, generate text informed by retrieved passages, and
    critique its own output using special reflection tokens. Self-RAG significantly
    outperforms existing RAG approaches and LLMs on multiple tasks. ArXiv ID: 2310.11511""",
]

sci_docs = DocumentLoader.from_texts(sample_scientific, "scientific", "demo_papers")
sci_chunks = chunker.chunk_documents(sci_docs, domain="scientific")
count = vs.add_documents("scientific", sci_chunks)
print(f"✅ Indexed {count} scientific chunks")

# Status check
print("\n📊 Index Status:")
for stat in vs.get_all_stats():
    print(f"  {stat['domain']:>12}: {stat['document_count']} chunks")

✅ Indexed 6 industrial chunks
✅ Indexed 3 recipe chunks
✅ Indexed 3 scientific chunks

📊 Index Status:
    industrial: 6 chunks
        recipe: 3 chunks
    scientific: 3 chunks


In [16]:
import sys

# Make sure the project root is at the FRONT of the path
PROJECT = '/content/drive/MyDrive/AI574_Multi_Domain_Agent/project'
if PROJECT not in sys.path:
    sys.path.insert(0, PROJECT)

# Verify the right modules are found
import agents.industrial_agent
print(f"✅ Found: {agents.industrial_agent.__file__}")

✅ Found: /content/drive/MyDrive/AI574_Multi_Domain_Agent/project/agents/industrial_agent.py


In [17]:
import importlib, orchestration.workflow_graph, orchestration.state_schema
importlib.reload(orchestration.state_schema)
importlib.reload(orchestration.workflow_graph)
from orchestration.workflow_graph import build_workflow, run_query

try:
    llm, vs, builder
except NameError:
    raise RuntimeError(
        "Run the cells above first so 'llm', 'vs', and 'builder' exist: "
        "load LLM, create embedder & vector store, run IndexBuilder and index data."
    )

workflow = build_workflow(llm=llm, vector_store=vs, index_builder=builder)
print("✅ Workflow compiled!")



✅ Workflow compiled!


In [33]:
def display_result(result):
    """Pretty-print query result with visual timing breakdown."""
    domain = result.get("domain", "?")
    conf   = result.get("confidence", 0)
    status = result.get("status", "?")
    esc    = result.get("escalated", False)
    srcs   = result.get("sources", [])

    print(f"\n{'='*60}")
    print(f"  Domain: {domain:<14} Confidence: {conf:.2f}")
    print(f"  Status: {status:<14} Escalated:  {esc}")
    print(f"  Sources: {len(srcs)}")
    print(f"{'='*60}")
    print(f"\n{result.get('response', '')}")

    # ── Timing ──────────────────────────────────────────────────
    t = result.get("timing")
    if not t:
        print("\n  (no timing data — run sync cell to update workflow_graph.py)")
        return

    total = t.get("total_s", 0) or 0.001
    sup   = t.get("supervisor_s", 0)
    agent = t.get("agent_s", 0)
    c     = t.get("crag") or {}

    retrieve = c.get("retrieve_s", 0)
    grade    = c.get("grade_s", 0)
    rewrite  = c.get("rewrite_s", 0)
    generate = c.get("generate_s", 0)
    validate = c.get("validate_s", 0)
    overhead = max(0, agent - (retrieve + grade + rewrite + generate + validate))

    steps = [
        ("Supervisor", sup),
        ("  Retrieve", retrieve),
        ("  Grade",    grade),
        ("  Rewrite",  rewrite),
        ("  Generate", generate),
        ("  Validate", validate),
        ("  Overhead", overhead),
    ]

    BAR = 25
    print(f"\n{'─'*60}")
    print(f"  TIMING BREAKDOWN  (total {total:.1f}s / {total/60:.2f} min)")
    print(f"{'─'*60}")
    for label, s in steps:
        if s < 0.01 and label == "  Rewrite":
            continue
        pct  = (s / total) * 100
        fill = int(BAR * s / total)
        bar  = "█" * fill + "░" * (BAR - fill)
        print(f"  {label:<12} {bar} {s:5.1f}s  ({pct:4.1f}%)")
    print(f"{'─'*60}")
    print(f"  {'Total':<12} {'':>{BAR}} {total:5.1f}s")
    print(f"{'─'*60}")

# ── Run test query ──────────────────────────────────────────────────────────
result = run_query(workflow, "My PowerFlex 525 drive is showing fault F004, how do I fix it?")
display_result(result)

INFO:foundation.llm_wrapper:LLM call: 1.1s | ~27 output tokens | ~24.2 tok/s
INFO:rag_core.crag_pipeline:CRAG attempt 1/3 | Query: My PowerFlex 525 drive is showing fault F004, how do I fix it?
INFO:foundation.llm_wrapper:LLM call: 9.7s | ~1313 output tokens | ~135.1 tok/s
INFO:foundation.llm_wrapper:LLM call: 2.2s | ~70 output tokens | ~31.3 tok/s
INFO:foundation.llm_wrapper:LLM call: 2.5s | ~88 output tokens | ~35.3 tok/s
INFO:foundation.llm_wrapper:LLM call: 2.2s | ~62 output tokens | ~28.0 tok/s
INFO:foundation.llm_wrapper:LLM call: 2.1s | ~61 output tokens | ~29.1 tok/s
INFO:foundation.llm_wrapper:LLM call: 2.0s | ~54 output tokens | ~27.5 tok/s
INFO:rag_core.crag_pipeline:Sufficient context found: 2 docs
INFO:foundation.llm_wrapper:LLM call: 12.4s | ~396 output tokens | ~32.1 tok/s
INFO:foundation.llm_wrapper:LLM call: 1.4s | ~15 output tokens | ~10.7 tok/s
INFO:rag_core.crag_pipeline:Response validated (confidence=1.00)
INFO:rag_core.crag_pipeline:⏱ CRAG timing: retrieve=0.0s gr


  Domain: industrial     Confidence: 0.97
  Status: complete       Escalated:  False
  Sources: 2

Okay, here's a troubleshooting guide for Fault Code F004 (Undervoltage) on your Allen-Bradley PowerFlex 525 drive, based solely on Document 1.

**Fault Code:** F004 - Undervoltage

**Description:** The DC bus voltage has dropped below the undervoltage trip level.

**Possible Causes (Document 1):**

1.  Input power supply voltage too low or momentary power dip.
2.  Incoming power supply fuses blown or circuit breaker tripped.
3.  Loose connections on input power terminals R/L1, S/L2, T/L3.
4.  DC bus capacitors degraded (check capacitor health indicator LED).

**Troubleshooting Steps (Document 1):**

1.  **Measure Incoming Line Voltage:** Measure the incoming line voltage at the drive terminals. It must be within the nameplate rating +/- 10%.
2.  **Check Voltage Balance:** Check all three phases for voltage balance (maximum 3% imbalance).
3.  **Inspect and Tighten Connections:** Inspect a

In [30]:
# Test: Recipe query
result = run_query(workflow, "What can I substitute for eggs in cookies?")
display_result(result)

INFO:foundation.llm_wrapper:LLM call: 1.2s | ~26 output tokens | ~22.1 tok/s
INFO:rag_core.crag_pipeline:CRAG attempt 1/3 | Query: What can I substitute for eggs in cookies?
INFO:foundation.llm_wrapper:LLM call: 4.5s | ~133 output tokens | ~29.4 tok/s
INFO:rag_core.crag_pipeline:Sufficient context found: 1 docs
INFO:foundation.llm_wrapper:LLM call: 5.7s | ~218 output tokens | ~38.5 tok/s
INFO:foundation.llm_wrapper:LLM call: 0.9s | ~15 output tokens | ~15.9 tok/s
INFO:rag_core.crag_pipeline:Response validated (confidence=1.00)
INFO:rag_core.crag_pipeline:⏱ CRAG timing: retrieve=0.0s grade=4.5s rewrite=0.0s generate=5.7s validate=0.9s total=11.1s (attempts=1)



  Domain: recipe         Confidence: 0.97
  Status: complete       Escalated:  False
  Sources: 1

Okay, I can help with that!

According to the Classic Chocolate Chip Cookie recipe, you have a few options for substituting eggs:

*   **Applesauce:** Use 1/4 cup of unsweetened applesauce per egg. Applesauce works as an egg substitute because it adds moisture and acts as a binder, similar to how eggs function.
*   **Mashed Banana:** Use 1 mashed banana per egg. Like applesauce, mashed banana provides moisture and binding properties. It will impart a slight banana flavor to your cookies.
*   **Aquafaba:** Use 3 tablespoons of aquafaba (the liquid from canned chickpeas) per egg. Aquafaba is a surprisingly effective binder and can help create a lighter texture.

**Recipe Summary (Classic Chocolate Chip Cookies):**

*   **Key Ingredients:** Flour, butter, sugar, eggs (or substitutes!), vanilla, chocolate chips.
*   **Prep Time:** 45 minutes
*   **Difficulty:** Easy

────────────────────────

In [28]:
# Test: Scientific query
result = run_query(workflow, "Summarize the key ideas behind RAG in NLP")
display_result(result)


INFO:foundation.llm_wrapper:LLM call: 1.3s | ~29 output tokens | ~21.9 tok/s
INFO:rag_core.crag_pipeline:CRAG attempt 1/3 | Query: Summarize the key ideas behind RAG in NLP
INFO:foundation.llm_wrapper:LLM call: 5.3s | ~169 output tokens | ~31.7 tok/s
INFO:rag_core.crag_pipeline:Sufficient context found: 2 docs
INFO:foundation.llm_wrapper:LLM call: 12.3s | ~511 output tokens | ~41.5 tok/s
INFO:foundation.llm_wrapper:LLM call: 1.3s | ~16 output tokens | ~12.2 tok/s
INFO:rag_core.crag_pipeline:Response validated (confidence=0.95)
INFO:rag_core.crag_pipeline:⏱ CRAG timing: retrieve=0.0s grade=5.3s rewrite=0.0s generate=12.3s validate=1.3s total=19.0s (attempts=1)



  Domain: scientific     Confidence: 0.97
  Status: complete       Escalated:  False
  Sources: 2

Here's a summary of the key ideas behind Retrieval-Augmented Generation (RAG) in NLP, based on the provided papers:

**Document 1: Self-RAG (Asai et al., 2023)**

*   **Objective:** To develop a framework (Self-RAG) that trains a single language model (LM) to perform adaptive retrieval of passages, generate text based on those passages, and critique its own output through self-reflection.
*   **Method:** Self-RAG trains an LM to simultaneously retrieve relevant passages, generate text incorporating the retrieved information, and evaluate its own generated text using special "reflection tokens."
*   **Key Findings:** Self-RAG demonstrates superior performance compared to existing RAG approaches and large language models (LLMs) across multiple tasks.
*   **Limitations:** The abstract does not detail specific tasks or the nature of the "reflection tokens."

**Document 2: Retrieval-Augmented

In [27]:
# Test: Ambiguous query (should trigger clarification)
result = run_query(workflow, "What temperature should I set to avoid failure?")
display_result(result)

INFO:foundation.llm_wrapper:LLM call: 1.3s | ~35 output tokens | ~25.9 tok/s
INFO:foundation.llm_wrapper:LLM call: 1.2s | ~39 output tokens | ~32.5 tok/s



  Domain: clarify        Confidence: 0.45
  Status: complete       Escalated:  False
  Sources: 0

"To help me give you the best advice, could you tell me what you're trying to keep at a certain temperature? (Like cooking, a car engine, or something else?)"

────────────────────────────────────────────────────────────
  TIMING BREAKDOWN  (total 2.6s / 0.04 min)
────────────────────────────────────────────────────────────
  Supervisor   █████████████░░░░░░░░░░░░   1.4s  (52.9%)
    Retrieve   ░░░░░░░░░░░░░░░░░░░░░░░░░   0.0s  ( 0.0%)
    Grade      ░░░░░░░░░░░░░░░░░░░░░░░░░   0.0s  ( 0.0%)
    Generate   ░░░░░░░░░░░░░░░░░░░░░░░░░   0.0s  ( 0.0%)
    Validate   ░░░░░░░░░░░░░░░░░░░░░░░░░   0.0s  ( 0.0%)
    Overhead   ░░░░░░░░░░░░░░░░░░░░░░░░░   0.0s  ( 0.0%)
────────────────────────────────────────────────────────────
  Total                                    2.6s
────────────────────────────────────────────────────────────


## Why does one query take 3+ minutes? / LLM throughput on A100

**Reasons:**
1. **Multiple LLM calls per query** — Supervisor (route) + batch grader + response generator + hallucination check = **4 sequential calls**. Each call is a full 9B forward/generation.
2. **Keras/JAX is not an inference-optimized stack** — Frameworks like TensorRT-LLM or vLLM get **~50–60 tok/s** for 8–10B on A100. Keras Hub on JAX typically does **~5–20 tok/s** (no fused kernels, no continuous batching).
3. **Rough token budget per query** — ~80 (supervisor) + ~300 (grader) + ~400–800 (generator) + ~80 (hallucination) ≈ **900–1300 output tokens**. At 10 tok/s that’s 90–130 s just for generation; add overhead and you get 3+ min.

**To see your actual throughput:** run the cell below and check the log lines `LLM call: Xs | ~Y output tokens | ~Z tok/s`.

In [35]:
# Enable INFO logging so "LLM call: ... tok/s" lines appear
import logging
logging.basicConfig(level=logging.INFO, format="%(message)s")
for _ in ["foundation.llm_wrapper", "rag_core.crag_pipeline"]:
    logging.getLogger(_).setLevel(logging.INFO)

# Timed re-run with full breakdown
result = run_query(workflow, "My PowerFlex 525 drive is showing fault F002, how do I fix it?")
display_result(result)

INFO:foundation.llm_wrapper:LLM call: 1.1s | ~27 output tokens | ~24.3 tok/s
INFO:rag_core.crag_pipeline:CRAG attempt 1/3 | Query: My PowerFlex 525 drive is showing fault F002, how do I fix it?
INFO:foundation.llm_wrapper:LLM call: 17.9s | ~1545 output tokens | ~86.4 tok/s
INFO:rag_core.crag_pipeline:Sufficient context found: 1 docs
INFO:foundation.llm_wrapper:LLM call: 7.1s | ~280 output tokens | ~39.5 tok/s
INFO:foundation.llm_wrapper:LLM call: 1.0s | ~15 output tokens | ~14.3 tok/s
INFO:rag_core.crag_pipeline:Response validated (confidence=1.00)
INFO:rag_core.crag_pipeline:⏱ CRAG timing: retrieve=0.0s grade=17.9s rewrite=0.0s generate=7.1s validate=1.0s total=26.0s (attempts=1)



  Domain: industrial     Confidence: 0.97
  Status: complete       Escalated:  False
  Sources: 1

Okay, here's a troubleshooting guide for Fault Code F002 on your Allen-Bradley PowerFlex 525 VFD, based solely on the provided document (rockwell_automation).

**Fault Code:** F002 - Auxiliary Input

**Description:** An external fault has been detected via the auxiliary input terminal.

**Troubleshooting Steps (from Document 1):**

1.  **Check the external device:** Verify the external device connected to the auxiliary input for any fault conditions.
2.  **Verify Wiring:** Check the wiring connections on terminal block TB2.
3.  **Check Parameter Configuration:** Check parameter A070 [Fault Config 2] for correct auxiliary input configuration.
4.  **Inspect Wiring:** Inspect for damaged or loose wiring on the digital input terminals.

**Resolution (from Document 1):**

*   Clear the external fault condition.
*   Reset the drive by cycling the Enable input or pressing the Stop/Reset key on 